In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns
import os
import spacy

In [ ]:
df1 = pd.read_csv('/Users/jaideepsai/Desktop/DATA-ANALYTICS/Data Source/extracted-data/Education/job_postings.csv')
df2 = pd.read_csv('/Users/jaideepsai/Desktop/DATA-ANALYTICS/Data Source/extracted-data/Education/job_skills.csv')
df3 = pd.read_csv('/Users/jaideepsai/Desktop/DATA-ANALYTICS/Data Source/extracted-data/Education/job_summary.csv')

In [ ]:
df = pd.merge(df1, df2, on='job_link')
df= pd.merge(df, df3, on='job_link')

In [ ]:
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_sm")

def extract_salary_with_nlp(description):
    doc = nlp(description)
    salaries = [ent.text for ent in doc.ents if ent.label_ == "MONEY"]
    return salaries if salaries else "Salary not specified"

def salary_to_int(salary_list):
    hours_per_week = 40
    weeks_per_year = 52
    results = []

    for salary_str in salary_list:
        salary_str = salary_str.replace(',', '').lower()
        if 'k' in salary_str:
            salary_str = salary_str.replace('k', '000')
            if '-' in salary_str:
                try:
                    low, high = [int(s.replace('$', '')) for s in salary_str.split('-')]
                    average_salary = (low + high) // 2
                    if average_salary < 1000000:
                        results.append(average_salary)
                except:
                    continue
            else:
                try:
                    if int(salary_str.replace('$', '')) < 1000000:
                        results.append(int(salary_str.replace('$', '')))
                except:
                    continue
                    
        elif '/hr' in salary_str:
            try:
                hourly_rate = int(salary_str.replace('$', '').split('/')[0])
            except:
                continue
                
            if hourly_rate > 100:
                results.append(hourly_rate)
            else:
                annual_salary = hourly_rate * hours_per_week * weeks_per_year
                if annual_salary < 1000000:
                    results.append(annual_salary)
                
        elif any(char.isdigit() for char in salary_str):  # Check if there's any digit
            try:
                if int(salary_str) < 100:
                    results.append(int(salary_str) * hours_per_week * weeks_per_year)
                else:
                    results.append(int(salary_str))
            except:
                continue
            
        else:
            try:
                results.append(int(salary_str.replace('$', '')))
            except ValueError:
                results.append(0)
                
    return sum(results) // len(results) if results else 0

def mark_positions_by_keyword(df, column_name, keywords, new_column_names):
    for i in range(len(keywords)):
        df[new_column_names[i]] = df[column_name].str.contains(keywords[i], case=False, na=False)
        
    return df

def extract_state(location):
    try:
        parts = location.split(', ')
        if len(parts) >= 2:
            return parts[1]  
        else:
            return None 
    except:
        return None
    
def extract_city(location):
    try:
        parts = location.split(', ')
        if len(parts) >= 2:
            return parts[1]  
        else:
            return None 
    except:
        return None

In [ ]:
df = mark_positions_by_keyword(df, 'job_title', ['Staff', 'Manager', 'Senior', 'Director', 'Software Engineer', 'Data Engineer', 'Machine Learning Engineer', 'Researcher'], 
                                   ['is_staff', 'is_manager', 'is_senior', 'is_director', 'is_swe', 'is_data_eng', 'is_mle', 'is_researcher'])

In [ ]:
df['state'] = df['job_location'].apply(extract_state)

In [ ]:
from tqdm import tqdm

tqdm.pandas(desc="My Progress Bar")

df['extracted_salaries'] = df['job_summary'].progress_apply(extract_salary_with_nlp)
df['annual_salary'] = df['extracted_salaries'].progress_apply(salary_to_int)
df_sh = df[df['annual_salary'] != 0.0]

In [ ]:
top_10 = df['search_country'].value_counts().nlargest(10)
top_20_cities = df['search_city'].value_counts().nlargest(25)
top_15_positions = df['job_title'].value_counts().nlargest(15)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
colors = plt.cm.viridis(top_10 / top_10.max())
plt.bar(top_10.index, top_10.values, color=colors)
plt.xlabel('Countries')
plt.ylabel('Frequency')
plt.title('Countries with the job openings in the dataset')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
colors = plt.cm.viridis(top_20_cities / top_20_cities.max())
plt.bar(top_20_cities.index, top_20_cities.values, color=colors)
plt.xlabel('Cities')
plt.ylabel('Frequency')
plt.title('Cities with the job openings in the dataset')
plt.xticks(rotation=45)

plt.show()

In [ ]:
import matplotlib.pyplot as plt

top_15_states= df['state'].value_counts().nlargest(15)

plt.figure(figsize=(20, 10))
colors = plt.cm.viridis(top_15_states / top_15_states.max())
plt.bar(top_15_states.index, top_15_states.values, color=colors)
plt.xlabel('States')
plt.ylabel('Frequency')
plt.title('States with the job openings in the dataset')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
colors = plt.cm.viridis(top_15_positions / top_15_positions.max())
plt.bar(top_15_positions.index, top_15_positions.values, color=colors)
plt.xlabel('Job positions')
plt.ylabel('Frequency')
plt.title('Job positions Distribution')
plt.xticks(rotation=45)

plt.show()

In [ ]:
import matplotlib.pyplot as plt

lst = ['is_staff', 'is_manager', 'is_senior', 'is_director', 'is_swe', 'is_data_eng', 'is_mle', 'is_researcher']


def viz_values_distrib(field_name):
    unique, counts = np.unique(df[field_name], return_counts=True)

    plt.figure(figsize=(20, 10))
    plt.bar(unique.astype(str), counts, color=['blue', 'orange'])
    plt.xlabel(field_name)
    plt.ylabel('Frequency')
    plt.title(field_name)
    plt.xticks(ticks=np.arange(len(unique)), labels=unique.astype(str))
    plt.show()
    
for i in range(len(lst)):
    viz_values_distrib(lst[i])


In [ ]:
import matplotlib.pyplot as plt

top_20_companies = df['company'].value_counts().nlargest(30)

plt.figure(figsize=(20, 10))
colors = plt.cm.viridis(top_20_companies / top_20_companies.max())
plt.bar(top_20_companies.index, top_20_companies.values, color=colors)
plt.xlabel('Companies')
plt.ylabel('Frequency')
plt.title('Companies with the job openings in the dataset')
plt.xticks(rotation=70)

plt.show()


In [ ]:
import matplotlib.pyplot as plt

opening_types = df['job_type'].value_counts().nlargest(30)

plt.figure(figsize=(20, 10))
colors = plt.cm.viridis(opening_types / opening_types.max())
plt.bar(opening_types.index, opening_types.values, color=colors)
plt.xlabel('Regime')
plt.ylabel('Types of work regime')
plt.title('Types of work regime (Remote / Onsite / Hybrid)')
plt.xticks(rotation=45)

plt.show()


In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(20, 10))

top_10_means = df_sh[df_sh['search_country'] == 'United States'].groupby('search_city')['annual_salary'].mean().sort_values(ascending=False).head(25)

colors = plt.cm.viridis(top_10_means / top_10_means.max())

top_10_means.plot(kind='bar', color=colors)
plt.xlabel('Cities')
plt.ylabel('$$$')
plt.title('TC by cities')
plt.xticks(rotation=45)
plt.show()

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(20, 10))

top_10_means = df_sh[df_sh['search_country'] == 'United States'].groupby('state')['annual_salary'].mean().sort_values(ascending=False).head(20)

colors = plt.cm.viridis(top_10_means / top_10_means.max())

top_10_means.plot(kind='bar', color=colors)
plt.xlabel('States')
plt.ylabel('$$$')
plt.title('TC by States')
plt.xticks(rotation=45)
plt.show()

In [ ]:
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np  # Import numpy for NaN handling

# Check for NaN or float values and replace them with an empty string
df['job_skills'] = df['job_skills'].apply(lambda x: '' if pd.isna(x) or isinstance(x, float) else x)

# Count skills frequency in the entire dataset
skill_counts = Counter()
for skills in df['job_skills'].str.split(','):
    skill_counts.update(skills)

# Create DataFrame with skill frequencies
skills_frequency = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Frequency'])
skills_frequency = skills_frequency.sort_values(by='Frequency', ascending=False).reset_index(drop=True).head(10)

# Plot skill frequencies
plt.figure(figsize=(20, 6))
colors = plt.cm.viridis(skills_frequency['Frequency'] / skills_frequency['Frequency'].max())
plt.bar(skills_frequency['Skill'], skills_frequency['Frequency'], color=colors)
plt.xlabel('Skills')
plt.ylabel('Frequency')
plt.title('General Skill Frequencies')
plt.xticks(rotation=45, ha='right')  # Rotate skill names for better readability
plt.show()


In [ ]:
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt

# Part 2: Skill Frequencies for Staff Job Position
skill_counts = Counter()
senior_df = df[df['is_staff'] == True]
for skills in senior_df['job_skills'].str.split(','):
    skill_counts.update(skills)
skills_frequency = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Frequency'])
skills_frequency = skills_frequency.sort_values(by='Frequency', ascending=False).reset_index(drop=True).head(10)

plt.figure(figsize=(20, 6))
colors = plt.cm.viridis(skills_frequency['Frequency'] / skills_frequency['Frequency'].max())
plt.bar(skills_frequency['Skill'], skills_frequency['Frequency'], color=colors)
plt.xlabel('Skills')
plt.ylabel('Frequency')
plt.title('Skill Frequencies for Staff Job Position')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
skill_counts = Counter()
senior_df = df[df['is_senior'] == True]
for skills in senior_df['job_skills'].str.split(','):
    skill_counts.update(skills)
skills_frequency = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Frequency'])
skills_frequency = skills_frequency.sort_values(by='Frequency', ascending=False).reset_index(drop=True).head(10)

plt.figure(figsize=(20, 6))
colors = plt.cm.viridis(skills_frequency['Frequency'] / skills_frequency['Frequency'].max())
plt.bar(skills_frequency['Skill'], skills_frequency['Frequency'], color=colors)
plt.xlabel('Skills')
plt.ylabel('Frequency')
plt.title('Skill Frequencies for Senior Job Position')
plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
skill_counts = Counter()
senior_df = df[df['is_director'] == True]
for skills in senior_df['job_skills'].str.split(','):
    skill_counts.update(skills)
skills_frequency = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Frequency'])
skills_frequency = skills_frequency.sort_values(by='Frequency', ascending=False).reset_index(drop=True).head(10)

plt.figure(figsize=(20, 6))
colors = plt.cm.viridis(skills_frequency['Frequency'] / skills_frequency['Frequency'].max())
plt.bar(skills_frequency['Skill'], skills_frequency['Frequency'], color=colors)
plt.xlabel('Skills')
plt.ylabel('Frequency')
plt.title('Skill Frequencies for Director Job Position')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
skill_counts = Counter()
senior_df = df[df['is_manager'] == True]
for skills in senior_df['job_skills'].str.split(','):
    skill_counts.update(skills)
skills_frequency = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Frequency'])
skills_frequency = skills_frequency.sort_values(by='Frequency', ascending=False).reset_index(drop=True).head(10)

plt.figure(figsize=(20, 6))
colors = plt.cm.viridis(skills_frequency['Frequency'] / skills_frequency['Frequency'].max())
plt.bar(skills_frequency['Skill'], skills_frequency['Frequency'], color=colors)
plt.xlabel('Skills')
plt.ylabel('Frequency')
plt.title('Skill Frequencies for Manager Job Position')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
skill_counts = Counter()
senior_df = df[df['is_swe'] == True]
for skills in senior_df['job_skills'].str.split(','):
    skill_counts.update(skills)
skills_frequency = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Frequency'])
skills_frequency = skills_frequency.sort_values(by='Frequency', ascending=False).reset_index(drop=True).head(10)

plt.figure(figsize=(20, 6))
colors = plt.cm.viridis(skills_frequency['Frequency'] / skills_frequency['Frequency'].max())
plt.bar(skills_frequency['Skill'], skills_frequency['Frequency'], color=colors)
plt.xlabel('Skills')
plt.ylabel('Frequency')
plt.title('Skill Frequencies for SWE Job Position')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
skill_counts = Counter()
senior_df = df[df['is_mle'] == True]
for skills in senior_df['job_skills'].str.split(','):
    skill_counts.update(skills)
skills_frequency = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Frequency'])
skills_frequency = skills_frequency.sort_values(by='Frequency', ascending=False).reset_index(drop=True).head(10)

plt.figure(figsize=(20, 6))
colors = plt.cm.viridis(skills_frequency['Frequency'] / skills_frequency['Frequency'].max())
plt.bar(skills_frequency['Skill'], skills_frequency['Frequency'], color=colors)
plt.xlabel('Skills')
plt.ylabel('Frequency')
plt.title('Skill Frequencies for MLE Job Position')
plt.xticks(rotation=45, ha='right')
plt.show()